In [1]:
import os
import json
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Get Data

In [74]:
ticker = 'AAPL'
stock = yf.Ticker(ticker)
data = stock.history(period="1mo", interval="30m")
data = data.reset_index(drop=False)
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-11-07 09:30:00-05:00,224.625000,226.550003,224.570007,226.300003,6727959,0.0,0.0
1,2024-11-07 10:00:00-05:00,226.300003,226.350006,225.360107,225.733597,3619511,0.0,0.0
2,2024-11-07 10:30:00-05:00,225.750000,226.229996,225.350006,226.229996,2005427,0.0,0.0
3,2024-11-07 11:00:00-05:00,226.229996,226.419998,225.785004,226.304993,1375092,0.0,0.0
4,2024-11-07 11:30:00-05:00,226.300003,227.100006,226.169998,227.070007,1865232,0.0,0.0


In [75]:
assert (data.columns.to_list() == ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'])
data.to_csv(f'../data/{ticker}_data.csv', index=False)

## Eda

In [186]:
file_name = f'../data/{ticker}_data.csv'
data = pd.read_csv(file_name, parse_dates=False)

data['Datetime'] = pd.to_datetime(data['Datetime'])
data['Datetime'] = data['Datetime'].dt.tz_localize(None)

data.Datetime.min(), data.Datetime.max()

(Timestamp('2024-11-07 09:30:00'), Timestamp('2024-12-06 15:30:00'))

In [187]:
data['MA_5'] = data['Close'].rolling(window=5).mean()
data['MA_30'] = data['Close'].rolling(window=30).mean()
data['MA_60'] = data['Close'].rolling(window=60).mean()

delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data.head(9)

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA_5,MA_30,MA_60,RSI
0,2024-11-07 09:30:00,224.625000,226.550003,224.570007,226.300003,6727959,0.0,0.0,NaN,NaN,NaN,NaN
1,2024-11-07 10:00:00,226.300003,226.350006,225.360107,225.733597,3619511,0.0,0.0,NaN,NaN,NaN,NaN
2,2024-11-07 10:30:00,225.750000,226.229996,225.350006,226.229996,2005427,0.0,0.0,NaN,NaN,NaN,NaN
3,2024-11-07 11:00:00,226.229996,226.419998,225.785004,226.304993,1375092,0.0,0.0,NaN,NaN,NaN,NaN
4,2024-11-07 11:30:00,226.300003,227.100006,226.169998,227.070007,1865232,0.0,0.0,226.327719,NaN,NaN,NaN
5,2024-11-07 12:00:00,227.061996,227.419998,226.759995,227.214996,1582418,0.0,0.0,226.510718,NaN,NaN,NaN
6,2024-11-07 12:30:00,227.210007,227.490005,227.119995,227.199997,1287558,0.0,0.0,226.803998,NaN,NaN,NaN
7,2024-11-07 13:00:00,227.205002,227.229996,226.770004,227.103302,1310965,0.0,0.0,226.978659,NaN,NaN,NaN
8,2024-11-07 13:30:00,227.100006,227.259995,226.744995,227.098907,1000216,0.0,0.0,227.137442,NaN,NaN,NaN


# Algo

In [188]:
data['Signal'] = 0
position = 0
holding_start = None

for i in range(1, len(data)):
    if (data['MA_5'].iloc[i] >= data['MA_30'].iloc[i]) and position == 0:
        position = 1
        data.at[data.index[i], 'Signal'] = position
        holding_start = data.at[data.index[i], 'Datetime']
    elif (data['MA_5'].iloc[i] < data['MA_30'].iloc[i]) and position == 1:
        position = 0
        data.at[data.index[i], 'Signal'] = -1
        if holding_start is not None:
            data.at[data.index[i], 'Hold'] = holding_start.strftime('%Y-%m-%d %H:%M:%S')
            holding_start = None
    elif (data['MA_5'].iloc[i] >= data['MA_30'].iloc[i]) and position == 1:
        data.at[data.index[i], 'Signal'] = 2
    elif (data['MA_5'].iloc[i] < data['MA_30'].iloc[i]) and position == 0:
        pass

data.iloc[100:105]

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA_5,MA_30,MA_60,RSI,Signal,Hold
100,2024-11-18 14:00:00,229.440002,229.440002,228.696106,228.815002,1252555,0.0,0.0,229.116040,226.853593,225.978326,75.085178,2,NaN
101,2024-11-18 14:30:00,228.820007,229.080307,228.720001,228.854996,1452563,0.0,0.0,229.137039,226.884593,226.049242,77.682606,2,NaN
102,2024-11-18 15:00:00,228.850006,228.910004,228.033707,228.059998,1950664,0.0,0.0,228.828638,226.870833,226.098075,69.236121,2,NaN
103,2024-11-18 15:30:00,228.050003,228.240005,227.320007,227.990005,3614507,0.0,0.0,228.633200,226.868500,226.148482,68.746481,2,NaN
104,2024-11-19 09:30:00,226.794998,228.550003,226.660004,228.434998,4341628,0.0,0.0,228.431000,226.865417,226.223732,69.933974,2,NaN


In [189]:
total_trades = 0
deals = []
hold_periods = []

for i in range(1, len(data)):
    if data['Signal'].iloc[i] == 1:
        deals.append({
            "date": data['Datetime'].iloc[i],
            "action": "Buy",
            "price": data['Open'].iloc[i]
        })
    elif data['Signal'].iloc[i] == -1:
        deals.append({
            "date": data['Datetime'].iloc[i],
            "action": "Sell",
            "price": data['Close'].iloc[i]
        })
        total_trades += 1
        if 'Hold' in data.columns and pd.notna(data['Hold'].iloc[i]):
            hold_periods.append({
                "start": data['Hold'].iloc[i],
                "end": data['Datetime'].iloc[i],
            })

data = data.replace({np.nan: None})
data.head(2)

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA_5,MA_30,MA_60,RSI,Signal,Hold
0,2024-11-07 09:30:00,224.625000,226.550003,224.570007,226.300003,6727959,0.0,0.0,None,None,None,None,0,None
1,2024-11-07 10:00:00,226.300003,226.350006,225.360107,225.733597,3619511,0.0,0.0,None,None,None,None,0,None


In [190]:
def count_profitable_trades(arr):
    i, j = 0, 1
    profitable_trades = 0
    budjet = 10_000 # dollars
    arr = arr[:-1] if len(arr) % 2 != 0 else arr

    while (i < len(arr)) and (j < len(arr)):
        amount_shares = budjet // float(arr[i]['price'])
        free_budjet = budjet % (float(arr[i]['price']) * amount_shares)
        price_delta = float(arr[j]['price']) - float(arr[i]['price'])
        if price_delta >= 0:
            profitable_trades += 1
        new_budjet = float(arr[j]['price']) * amount_shares + free_budjet

        print('Sell', float(arr[j]['price']), 'Buy', float(arr[i]['price']), budjet, new_budjet)
        
        i += 2
        j += 2
        budjet = new_budjet

    result = round(new_budjet - 10_000, 2)
    cumulative_return = round(result * 100 / 10_000, 2)

    return budjet, profitable_trades, result, cumulative_return

In [191]:
budjet_left, profitable_trades, outcome, cumulative_return = count_profitable_trades(deals)
win_rate = round(profitable_trades / total_trades, 2)

Sell 224.6199951171875 Buy 224.52999877929688 10000 10003.959838867188
Sell 226.51499938964844 Buy 228.1000061035156 10003.959838867188 9935.8045501709
Sell 242.8600006103516 Buy 228.8500061035156 9935.8045501709 10538.23431396485


In [192]:
budjet_left, profitable_trades, outcome, cumulative_return

(10538.23431396485, 2, 538.23, 5.38)